# Retrieving on demand features

## 1. Instantiate a `FeatureStore` object

In [2]:
from feast import FeatureStore
import pandas as pd
from datetime import datetime

In [3]:
store = FeatureStore(repo_path=".")

# 2. Retrieve historical features

### model_v2 feature service
This one leverages dummy `val_to_add` and `val_to_add_2` request data. Request data must be passed in as part of the `entity_df`. This may come from the same source that includes your labels for your model.

A quick reminder of the on demand feature view (ODFV) being used in this feature service:

In [12]:
import dill
print(dill.source.getsource(store.get_on_demand_feature_view("transformed_conv_rate").udf))

@on_demand_feature_view(
    sources=[driver_hourly_stats_view, val_to_add_request],
    schema=[
        Field(name="conv_rate_plus_val1", dtype=Float64),
        Field(name="conv_rate_plus_val2", dtype=Float64),
    ],
)
def transformed_conv_rate(inputs: pd.DataFrame) -> pd.DataFrame:
    df = pd.DataFrame()
    df["conv_rate_plus_val1"] = inputs["conv_rate"] + inputs["val_to_add"]
    df["conv_rate_plus_val2"] = inputs["conv_rate"] + inputs["val_to_add_2"]
    return df



Now let's retrieve historical features from this feature service. The transformation will happen on the fly after doing the point-in-time join to produce the `conv_rate_plus_val1` and `conv_rate_plus_val2` features

In [30]:
entity_df = pd.DataFrame.from_dict(
    {
        "driver_id": [1001, 1002, 1003, 1004],
        "event_timestamp": [
            datetime(2021, 4, 12, 10, 59, 42),
            datetime(2021, 4, 12, 8, 12, 10),
            datetime(2021, 4, 12, 16, 40, 26),
            datetime(2021, 4, 12, 15, 1, 12),
        ],
        "val_to_add": [1, 2, 3, 4],
        "val_to_add_2": [10, 20, 30, 40],
    }
)
training_df = store.get_historical_features(
    entity_df=entity_df,
    features=store.get_feature_service("model_v2"),
).to_df()
print(training_df.head())

      driver_id           event_timestamp  val_to_add  val_to_add_2  \
360        1001 2021-04-12 10:59:42+00:00           1            10   
721        1002 2021-04-12 08:12:10+00:00           2            20   
1084       1003 2021-04-12 16:40:26+00:00           3            30   
1445       1004 2021-04-12 15:01:12+00:00           4            40   

      conv_rate  conv_rate_plus_val1  conv_rate_plus_val2  
360    0.521149             1.521149            10.521149  
721    0.089014             2.089014            20.089014  
1084   0.188855             3.188855            30.188855  
1445   0.296492             4.296492            40.296492  


### model_v3 feature service
This one generates geohash features from latitudes and longitudes. This is useful for generating features relating to geographic regions (e.g. A geohash of `qfb9c3mw8hte` represents a sub-region within the region represented by a `qfb` geohash.)

Let's look at the on demand feature view used in this feature service:

In [13]:
print(dill.source.getsource(store.get_on_demand_feature_view("location_features_from_push").udf))

@on_demand_feature_view(
    sources=[driver_daily_features_view],
    schema=[Field(name=f"geohash_{i}", dtype=String) for i in range(1, 7)],
)
def location_features_from_push(inputs: pd.DataFrame) -> pd.DataFrame:
    import pygeohash as gh

    df = pd.DataFrame()
    df["geohash"] = inputs.apply(lambda x: gh.encode(x.lat, x.lon), axis=1).astype(
        "string"
    )

    for i in range(1, 7):
        df[f"geohash_{i}"] = df["geohash"].str[:i].astype("string")
    return df



Now we retrieve features. This will compute the `geohash_X` features

In [31]:
entity_df = pd.DataFrame.from_dict(
    {
        "driver_id": [1001, 1002, 1003, 1004],
        "event_timestamp": [
            datetime(2021, 4, 12, 10, 59, 42),
            datetime(2021, 4, 12, 8, 12, 10),
            datetime(2021, 4, 12, 16, 40, 26),
            datetime(2021, 4, 12, 15, 1, 12),
        ]
    }
)

training_df = store.get_historical_features(
    entity_df=entity_df,
    features=store.get_feature_service("model_v3"),
).to_df()
print(training_df.head())

      driver_id           event_timestamp  daily_miles_driven       lat  \
360        1001 2021-04-12 10:59:42+00:00           18.926695  1.265647   
721        1002 2021-04-12 08:12:10+00:00           12.005569  0.722192   
1084       1003 2021-04-12 16:40:26+00:00           23.490234  1.330712   
1445       1004 2021-04-12 15:01:12+00:00           19.204191  0.961260   

           lon       geohash geohash_1 geohash_2 geohash_3 geohash_4  \
360   1.150815  s00z4nmuzvtv         s        s0       s00      s00z   
721   0.290492  s00hne7x0fqj         s        s0       s00      s00h   
1084  2.996348  s04ps4jzgyxq         s        s0       s04      s04p   
1445  5.048517  s05t6yupwzyu         s        s0       s05      s05t   

     geohash_5 geohash_6  
360      s00z4    s00z4n  
721      s00hn    s00hne  
1084     s04ps    s04ps4  
1445     s05t6    s05t6y  


# 3. Retrieve online features

### model_v2 feature service
This one leverages dummy `val_to_add` and `val_to_add_2` request data so this is passed into the `entity_rows` parameter. The on demand transformation is executed on the fly and combines in this request data with the pre-computed `conv_rate` feature in the online store.

In [32]:
features = store.get_online_features(
    features=store.get_feature_service("model_v2"),
    entity_rows=[{"driver_id": 1001, "val_to_add": 1000, "val_to_add_2": 2000,}],
).to_dict()
for key, value in sorted(features.items()):
    print(key, " : ", value)

conv_rate  :  [0.4045884609222412]
conv_rate_plus_val1  :  [1000.4045884609222]
conv_rate_plus_val2  :  [2000.4045884609222]
driver_id  :  [1001]


### model_v3 feature service
This one generates geohash features from latitude and longitude values in the online store.

Note that this feature service relies on a `PushSource` so no lat / lon values are needed at request time. Perhaps there's a separate thread on the driver's app that asynchronously pushes the driver's location to a Kafka topic.

In [33]:
features = store.get_online_features(
    features=store.get_feature_service("model_v3"),
    entity_rows=[{"driver_id": 1001}],
).to_dict()
for key, value in sorted(features.items()):
    print(key, " : ", value)

daily_miles_driven  :  [350.6502685546875]
driver_id  :  [1001]
geohash_1  :  ['s']
geohash_2  :  ['s0']
geohash_3  :  ['s07']
geohash_4  :  ['s07z']
geohash_5  :  ['s07z6']
geohash_6  :  ['s07z6m']
lat  :  [2.71002197265625]
lon  :  [5.3769989013671875]
